In [ ]:
# Add src to Python path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

# Import framework components
from src.data.loaders import DataLoader
from src.data.processors import OptionsDataProcessor
from src.models.pricing import BlackScholes
from src.models.volatility import VolatilitySurface
from src.utils.plotting import plot_volatility_surface, plot_volatility_smile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Framework imported successfully!")

## 1. Create Sample Data

For this example, we'll create synthetic options data to demonstrate the framework.

In [ ]:
# Create sample options data
spot_price = 50000  # BTC spot price
risk_free_rate = 0.03

# Generate strikes around spot
strikes = np.linspace(40000, 60000, 15)
expiries = np.array([0.1, 0.25, 0.5, 1.0])  # 1M, 3M, 6M, 1Y

# Create options chain
options_data = []

for expiry in expiries:
    for strike in strikes:
        # Simulate implied volatility with smile
        moneyness = strike / spot_price
        # Simple smile: higher vol for OTM options
        base_vol = 0.80
        smile_factor = 0.2 * (moneyness - 1.0)**2
        iv = base_vol + smile_factor
        
        # Calculate call price
        call_price = BlackScholes.price(
            spot_price, strike, expiry, risk_free_rate, iv, "call"
        )
        
        options_data.append({
            'strike': strike,
            'expiry': expiry,
            'implied_vol': iv,
            'option_type': 'call',
            'price': call_price,
            'spot': spot_price
        })

df = pd.DataFrame(options_data)
print(f"Created {len(df)} sample options")
df.head()

## 2. Construct Volatility Surface

In [ ]:
# Build volatility surface
vol_surface = VolatilitySurface()

vol_surface.fit(
    df['strike'].values,
    df['expiry'].values,
    df['implied_vol'].values,
    method='rbf'
)

print("Volatility surface fitted!")

# Test interpolation
test_strike = 52000
test_expiry = 0.3

interpolated_iv = vol_surface.get_iv(test_strike, test_expiry)
print(f"\nInterpolated IV at K={test_strike}, T={test_expiry}: {interpolated_iv:.2%}")

## 3. Visualize Volatility Surface

In [ ]:
# Create grid for surface plot
K_grid = np.linspace(strikes.min(), strikes.max(), 30)
T_grid = np.linspace(expiries.min(), expiries.max(), 20)
K_mesh, T_mesh = np.meshgrid(K_grid, T_grid)

# Interpolate IV across grid
IV_mesh = np.zeros_like(K_mesh)
for i in range(K_mesh.shape[0]):
    for j in range(K_mesh.shape[1]):
        IV_mesh[i, j] = vol_surface.get_iv(K_mesh[i, j], T_mesh[i, j])

# Plot 3D surface
fig = plot_volatility_surface(
    K_mesh, T_mesh, IV_mesh, 
    spot=spot_price,
    figsize=(14, 8)
)
plt.show()

## 4. Analyze Volatility Smile

In [ ]:
# Plot smile for different expiries
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, expiry in enumerate(expiries):
    # Filter data for this expiry
    expiry_data = df[df['expiry'] == expiry]
    
    ax = axes[idx]
    ax.plot(expiry_data['strike'], expiry_data['implied_vol'], 'o-', linewidth=2)
    ax.axvline(spot_price, color='red', linestyle='--', alpha=0.5, label='Spot')
    ax.set_xlabel('Strike')
    ax.set_ylabel('Implied Volatility')
    ax.set_title(f'Volatility Smile (T = {expiry:.2f}y)')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

## 5. Calculate Greeks

Let's calculate option Greeks for risk analysis.

In [ ]:
from src.models.greeks import GreeksCalculator

# Calculate Greeks for ATM option
atm_strike = spot_price
expiry_6m = 0.5
vol_6m = vol_surface.get_iv(atm_strike, expiry_6m)

greeks = GreeksCalculator.all_greeks(
    S=spot_price,
    K=atm_strike,
    T=expiry_6m,
    r=risk_free_rate,
    sigma=vol_6m,
    option_type='call'
)

print("Greeks for ATM 6M Call:")
for greek, value in greeks.items():
    print(f"  {greek.capitalize()}: {value:.4f}")

## Next Steps

This notebook demonstrated the basic framework usage. Explore:

- `notebooks/exploratory/` - Data exploration and analysis
- `notebooks/modeling/` - Volatility model development
- `notebooks/backtesting/` - Strategy backtesting

Refer to the README.md for more information on the framework architecture.